In [ ]:
pip install psycopg2 pandas requests

In [ ]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import requests

languageCode = 'fr'
t_host = "DRAK" # either "localhost", a domain name, or an IP address.
t_port = "5435" # default postgres port
t_dbname = "postgres"
t_user = "postgres"
t_pw = "postgres"

input_data = None

with psycopg2.connect("host='{}' port={} dbname={} user={} password={}".format(t_host, t_port, t_dbname, t_user, t_pw)) as conn:
    
    sql_select_all = '''SELECT sub.id,  sub.geom_x,  sub.geom_y, sub.geom, sub.siren, sub.siret, sub.statutdiffusionetablissement, sub.adress, sub.libellevoieetablissement, sub.libellecommuneetablissement, 
                            COALESCE(sub.enseigne1etablissement, sub.enseigne2etablissement, sub.enseigne3etablissement) as enseigneetablissement, 
                            COALESCE(sub.denominationusuelleetablissement, sub.enseigne1etablissement, sub.enseigne2etablissement, sub.enseigne3etablissement) as denominationusuelleetablissement 
                            FROM (SELECT id, geom, siren, siret, statutdiffusionetablissement, 
                                UPPER(CONCAT_WS(' ', NULLIF(numerovoieetablissement::text, '[ND]'), NULLIF(typevoieetablissement::text, '[ND]'), NULLIF(libellevoieetablissement::text, '[ND]'), NULLIF(codepostaletablissement::text, '[ND]'), NULLIF(libellecommuneetablissement::text, '[ND]'))) as adress,
                                NULLIF(UPPER(libellevoieetablissement), '[ND]') as libellevoieetablissement, 
                                NULLIF(UPPER(libellecommuneetablissement), '[ND]') as libellecommuneetablissement, 
                                NULLIF(denominationusuelleetablissement, '[ND]') as denominationusuelleetablissement,
                                NULLIF(enseigne1etablissement, '[ND]') as enseigne1etablissement,
                                NULLIF(enseigne2etablissement, '[ND]') as enseigne2etablissement,
                                NULLIF(enseigne3etablissement, '[ND]') as enseigne3etablissement,
                                ST_X(ST_GeomFromEWKT(geom)) as geom_x,
                                ST_Y(ST_GeomFromEWKT(geom)) as geom_y
                               FROM siren.siren_archis) sub;'''
    input_data = pd.read_sql_query(sql_select_all, conn)
    



    

In [ ]:
from collections import OrderedDict
from operator import getitem
import json

url = "https://places.googleapis.com/v1/places:searchNearby"
headers = {"Content-type": "application/json", "X-Goog-Api-Key": "YOUR_GOOGLE_API_TOKEN", "X-Goog-FieldMask": "*"}

for index, data in input_data.iterrows():

    
    json_post_data = {
                        "maxResultCount": 3,
                        "rankPreference": "DISTANCE",
                        "locationRestriction": {
                            "circle": {
                            "center": {
                                "latitude": data['geom_y'],
                                "longitude": data['geom_x']},
                            "radius": 500.0
                            }
                        }
                    }
    r = requests.post(url, data=json.dumps(json_post_data), headers=headers)
    all_data_results = []
    
    if r.json() and str(r.json()) != '{}':
        request_results = r.json()
        
        
        
        data_adress_matching_array = data['adress'].replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip().split()
        
        
        for y, place_result in enumerate(request_results["places"]):
            data_result = {"ranking": {"adress_indice":0, "name_in_base": False, "same_names":False, "result_name":"", "same_language":False}, "place_result": place_result}
            
            print(y)
            
            
            if 'formattedAddress' in place_result.keys():
  
                adress_place_result = place_result['formattedAddress'].replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()
                
                
                
                found = 0
                for string in data_adress_matching_array:
                    if string in adress_place_result:
                        found += 1
                data_result["ranking"]["adress_indice"] = found
                
            
            if 'displayName' in place_result.keys():
                
                data_result["ranking"]["result_name"] = place_result['displayName']["text"].replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()
                    

                    

                if data['enseigneetablissement'] is not None:           
                    name_place_compare = data['enseigneetablissement'].replace(' ',' ').replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()
                    if name_place_compare == place_result["displayName"]["text"].replace(' ',''):
                       data_result["ranking"]["same_names"] = True
                else:
                    data_result["ranking"]["name_in_base"] = False
                                
                if data_result["ranking"]["same_names"] is False:    
                    if data['denominationusuelleetablissement'] is not None:
                        name_place_compare = data['denominationusuelleetablissement'].replace(' ',' ').replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()
                        if name_place_compare == place_result["displayName"]["text"].replace(' ',''):
                           data_result["ranking"]["same_names"] = True
                    else:
                        data_result["ranking"]["name_in_base"] = False
                        
            all_data_results.append(data_result)
                        
         
        row_data_results = sorted(all_data_results, key=lambda item: (item["ranking"]["same_names"] if item["ranking"]["name_in_base"] is True else True, item["ranking"]["adress_indice"]))

        print(json.dumps(row_data_results))

        sql_upsert = '''INSERT INTO siren.google_place
                            (siren_archis_fk, "result", complete_return)
                            VALUES ({siren_archis_fk}, {result}, {complete_return})
                            ON CONFLICT (siren_archis_fk)
                            DO UPDATE 
                            SET "result"= case
                                            when json_typeof(src."result") ilike 'array'
                                            then json_build_array(src."result" || EXCLUDED."result"),   
                                            when json_typeof(src."result") ilike 'object'
                                            then src."result", EXCLUDED."result", 
                                complete_return= src."complete_return" || EXCLUDED.complete_return;'''.format(siren_archis_fk=data['id'], result=json.dumps(row_data_results[0]), complete_return=json.dumps(row_data_results))    
        

        
        with psycopg2.connect("host='{}' port={} dbname={} user={} password={}".format(t_host, t_port, t_dbname, t_user, t_pw)) as conn:                       
            cur = conn.cursor()
            cur.execute(sql_upsert)
            conn.commit()
            cur.close()

        print(index, data_result[0])

                
        

In [ ]:
from cartoframes.auth import set_default_credentials
from cartoframes.data.services import Geocoding
from cartoframes.viz import Map, Layer, size_continuous_style

set_default_credentials('creds.json')

stores_df = pd.read_csv('http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn.csv')
stores_gdf, _ = Geocoding().geocode(stores_df, street='address')
Map(Layer(stores_gdf, size_continuous_style('revenue', size_range=[10,40]), title='Annual Revenue ($)'))